In [191]:
import pandas as pd

In [192]:
from pymorphy3 import MorphAnalyzer
from nltk.corpus import stopwords

morph = MorphAnalyzer()
stopwords_ru = stopwords.words("russian")

def morphe(str):
    return ' '.join([morph.normal_forms(w)[0] for w in str.split() if w not in stopwords_ru])

In [193]:
fillna_columns = [
    "занятость",
    "по должности-лемме",
    "по дополнительному признаку",
    "по условиям",
    "общие фразы",
]

In [194]:
df = pd.read_csv("answers.csv", index_col='Unnamed: 0')
for col in fillna_columns:
    df.fillna({col: f'{col}_нет'}, inplace=True)

df['query'] = df['query'].apply(morphe)

In [195]:
onehot_columns = ['по должности-лемме', 'по условиям', 'общие фразы']
multilabel_columns = ['занятость', 'по дополнительному признаку']
vectorizer_columns = 'query'

In [196]:
def split(x):
    return [i for i in set(x.split(',')) if len(i)>0]

for col in multilabel_columns:
    df[col] = df[col].apply(lambda x: split(x))

In [197]:
X = df[['query']]
y = df.drop(columns=['query'])

In [198]:
y

,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
0,[занятость_нет],по должности-лемме_нет,[по дополнительному признаку_нет],по условиям_нет,общая фраза
1,[занятость_нет],по должности-лемме_нет,[по дополнительному признаку_нет],по условиям_нет,общая фраза
2,[занятость_нет],Рабочий,[по дополнительному признаку_нет],по условиям_нет,общие фразы_нет
3,[на неполный день],Водитель,[по дополнительному признаку_нет],по условиям_нет,общие фразы_нет
4,[занятость_нет],Водитель,[по дополнительному признаку_нет],по условиям_нет,общие фразы_нет
...,...,...,...,...,...
14240,[занятость_нет],по должности-лемме_нет,[по дополнительному признаку_нет],по условиям_нет,общая фраза
14241,[занятость_нет],по должности-лемме_нет,[по дополнительному признаку_нет],по условиям_нет,общая фраза
14242,[занятость_нет],по должности-лемме_нет,[по дополнительному признаку_нет],по условиям_нет,общая фраза
14243,[занятость_нет],по должности-лемме_нет,[по дополнительному признаку_нет],по условиям_нет,общая фраза


In [199]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.compose import ColumnTransformer

from sklearn.feature_extraction.text import CountVectorizer


In [200]:
from sklearn.pipeline import FunctionTransformer


preprocessor_y = ColumnTransformer(
    transformers=[

        (
            "onehot",
            OneHotEncoder(sparse_output=False),
            onehot_columns,
        ),
        ('multilabel0', CountVectorizer(analyzer=set), multilabel_columns[0]),
        ("multilabel1", CountVectorizer(analyzer=set), multilabel_columns[1]),
        # ("vectorizer", CountVectorizer(), vectorizer_columns),
    ]
)

preprocessor_x = ColumnTransformer(transformers=[
    ('vectorize', CountVectorizer(), vectorizer_columns)
])

In [201]:
from sklearn.base import BaseEstimator, TransformerMixin


class InverseTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, pipe):
        self.pipe = pipe

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        inverse_transformers = self.pipe.transformers_

        len1 = len(inverse_transformers[0][1].get_feature_names_out())
        len2 = len(list(inverse_transformers[1][1].vocabulary_.keys()))

        res = inverse_transformers[0][1].inverse_transform([cut[:len1]])
        res2 = inverse_transformers[1][1].inverse_transform([cut[len1 : len1 + len2]])
        res3 = inverse_transformers[2][1].inverse_transform([cut[len1 + len2 :]])

        output = dict(zip(inverse_transformers[0][1].feature_names_in_, res[0]))
        output["занятость"] = res2[0].tolist()
        output["по дополнительному признаку"] = res3[0].tolist()
        return output

In [202]:
len(inverse_transformers[0][1].get_feature_names_out())

303

In [203]:
from sklearn.neural_network import MLPClassifier

In [204]:
# pipeline = Pipeline([
#     ("preprocessor", preprocessor),
#     ("model", MLPClassifier(max_iter=400)),
#     ("inverse", FunctionTransformer())
# ])

In [205]:
y_prep = preprocessor_y.fit_transform(y)
y_prep

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [206]:
# x_prep = preprocessor_x.fit_transform(X)
# x_prep.todense()

In [207]:
model = MLPClassifier(max_iter=400)

In [208]:
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor_x),
    ("model", model),
    # ("inverse", InverseTransformer(preprocessor_y))
])

In [209]:
pipeline.fit(X, y_prep)

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'MLPClassifier(max_iter=400)' (type <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>) doesn't

In [ ]:
example_qeury = "работа грузчиком с проживанием"
query_prep = morphe(example_qeury)

frame = pd.DataFrame({'query': [query_prep]})

In [ ]:
# example_prep = preprocessor_x.transform(frame)
# preds = model.predict(example_prep)
preds2 = pipeline.predict(frame)
# cut = preds[0]

In [ ]:
preds2

In [ ]:
inverser = InverseTransformer(preprocessor_y)
result = inverser.transform(preds2)

In [ ]:
# inverse_transformers = preprocessor_y.transformers_

In [ ]:
# list(inverse_transformers[1][1].vocabulary_.keys())

['занятость_нет',
 'на неполный день',
 'Подработка',
 'на дому',
 'Вахта',
 'Ночная',
 'по выходным',
 'Удаленная',
 'Вечерняя',
 'Временная',
 'Посменная',
 'Дневная',
 'Посуточная']

In [ ]:
# res = inverse_transformers[0][1].inverse_transform([cut[:303]])
# res

array([['Грузчик', 'с проживанием', 'общие фразы_нет']], dtype=object)

In [ ]:
# output = dict(zip(inverse_transformers[0][1].feature_names_in_, res[0]))

In [ ]:
# res2 = inverse_transformers[1][1].inverse_transform([cut[303:316]])
# output['занятость'] = res2[0].tolist()

In [ ]:
# res2

[array(['занятость_нет'], dtype='<U16')]

In [ ]:
# res3 = inverse_transformers[2][1].inverse_transform([cut[316:]])
# output['по дополнительному признаку'] = res3[0].tolist()

In [ ]:
# output

{'по должности-лемме': 'Грузчик',
 'по условиям': 'с проживанием',
 'общие фразы': 'общие фразы_нет',
 'занятость': ['занятость_нет'],
 'по дополнительному признаку': ['по дополнительному признаку_нет']}

In [ ]:
# import dill

# with open('model.pkl', 'wb') as file:
#     dill.dump(model, file)